<a href="https://colab.research.google.com/github/jiheddachraoui/occupancy_grid_generator/blob/main/bird_grid_saver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PREPARATION

In [ ]:
import os
import sys
import csv
import numpy as np
import cv2
import matplotlib.pyplot as plt
import time
import tensorflow as tf
from sklearn.linear_model import RANSACRegressor
from scipy import stats
from google.colab import drive
drive.mount('/content/drive',force_remount=True)


Mounted at /content/drive


In [ ]:
datadir = '/content/drive/MyDrive/s_data/'
scenario='ground truth/scenario4_gt/'


    
for item in os.listdir(os.path.join(datadir,scenario)):
    # Construct the full path to the item
    item_path = os.path.join(os.path.join(datadir,scenario), item)
    # Check if the item is a directory
    if os.path.isdir(item_path) and item != '.ipynb_checkpoints':
        # Print the path of the subdirectory
        print(item_path)
        


/content/drive/MyDrive/s_data/ground truth/scenario4_gt/Ego_1 vehicle.audi.a2
/content/drive/MyDrive/s_data/ground truth/scenario4_gt/Vehicle_2 vehicle.mini.cooper_s
/content/drive/MyDrive/s_data/ground truth/scenario4_gt/Vehicle_1 vehicle.tesla.model3


In [ ]:
vehicles=[]
for item in os.listdir(os.path.join(datadir,scenario)):
    # Construct the full path to the item
    if os.path.isdir(item_path) and item != '.ipynb_checkpoints':
        folder_path = os.path.join(os.path.join(datadir,scenario), item)
        lidar_path=folder_path+'/bird_lidar'
        img_path=folder_path+'/bird_eye'
        save_path=folder_path+'/bird_grid_gt'
        if not os.path.exists(save_path):
            os.makedirs(save_path)
        num_files = len([f for f in os.listdir(lidar_path) if os.path.isfile(os.path.join(lidar_path, f))])
        print('Number of files in {} :'.format(lidar_path), num_files)
        vehicles.append((lidar_path,num_files,save_path))

Number of files in /content/drive/MyDrive/s_data/ground truth/scenario4_gt/Ego_1 vehicle.audi.a2/bird_lidar : 89
Number of files in /content/drive/MyDrive/s_data/ground truth/scenario4_gt/Vehicle_2 vehicle.mini.cooper_s/bird_lidar : 89
Number of files in /content/drive/MyDrive/s_data/ground truth/scenario4_gt/Vehicle_1 vehicle.tesla.model3/bird_lidar : 89


In [ ]:
vehicles

[('/content/drive/MyDrive/s_data/ground truth/scenario4_gt/Ego_1 vehicle.audi.a2/bird_lidar',
  89,
  '/content/drive/MyDrive/s_data/ground truth/scenario4_gt/Ego_1 vehicle.audi.a2/bird_grid_gt'),
 ('/content/drive/MyDrive/s_data/ground truth/scenario4_gt/Vehicle_2 vehicle.mini.cooper_s/bird_lidar',
  89,
  '/content/drive/MyDrive/s_data/ground truth/scenario4_gt/Vehicle_2 vehicle.mini.cooper_s/bird_grid_gt'),
 ('/content/drive/MyDrive/s_data/ground truth/scenario4_gt/Vehicle_1 vehicle.tesla.model3/bird_lidar',
  89,
  '/content/drive/MyDrive/s_data/ground truth/scenario4_gt/Vehicle_1 vehicle.tesla.model3/bird_grid_gt')]

In [ ]:
ALPHA = 1
BHETA = 1*np.pi/180
RESOLUTION = 0.1
MAX_RANGE = 70
MAP_WIDTH = 100
SPHERICAL2CARTESIAN_BIAS = 6
### Some derived parameters
# OOR_MASK = A mask to filter out measurements that are out of MAX_RANGE
MAP_SIZE_X = int(MAP_WIDTH/RESOLUTION)
MAP_SIZE_Y = int(MAP_WIDTH/RESOLUTION)
xarr = np.arange(-MAP_WIDTH/2,MAP_WIDTH/2,RESOLUTION)
yarr = np.arange(-MAP_WIDTH/2,MAP_WIDTH/2,RESOLUTION)
MAP_XX, MAP_YY = np.meshgrid(xarr, -yarr)
rgrid = np.sqrt(np.add(np.square(MAP_XX),np.square(MAP_YY)))
OOR_MASK = rgrid >= MAX_RANGE

In [ ]:
def filter_road_points(lidar_in, is_road, threshold=0.5):
  y_true = lidar_in[:,1]
  y_pred = np.zeros_like(y_true)
  
  y_pred = y_true * is_road  # set road points' predicted height to true height
  delta_y = np.absolute(y_true-y_pred).flatten()
  is_not_road = delta_y > threshold
  lidar_out = lidar_in[is_not_road,:].copy()
  return lidar_out

In [ ]:
def generate_semantic_ogm(lidar_in, ogm_shape):
  ### Calculate the position of LiDAR points in spherical coordinate
    rphi_meas = np.zeros((lidar_in.shape[0], 3))
    rphi_meas[:, 1] = np.sqrt(np.add(np.square(lidar_in[:, 0]), np.square(lidar_in[:, 1]))) / ALPHA
    rphi_meas[:, 0] = (np.arctan2(lidar_in[:, 1], lidar_in[:, 0]) + np.pi) / BHETA
    rphi_meas[:, 2]=lidar_in[:,2]
    
    rphi_meas = np.unique(rphi_meas.astype(int), axis=0)
    rphi_meas = rphi_meas[rphi_meas[:, 1] < int(MAX_RANGE / ALPHA), :]
    rphi_meas = rphi_meas[rphi_meas[:, 0] < int(2 * np.pi / BHETA), :]

    ### Initiate and fill the spherical scan grid
    sg_ang_bin = int(2 * np.pi / BHETA)
    sg_rng_bin = int(MAX_RANGE / ALPHA)
    # Initiation (Condition 3)
    
    scan_grid = np.ones((sg_ang_bin, sg_rng_bin)) * 0.5
    label_grid = np.zeros((sg_ang_bin, sg_rng_bin))
    # Condition 1
    
    scan_grid[tuple(rphi_meas[:,[0,1]].T)] = 0.7
    for i in range(rphi_meas.shape[0]):
      ang = rphi_meas[i, 0]
      rng = rphi_meas[i, 1]
      label = rphi_meas[i, 2]
      label_grid[int(ang), int(rng)] = label
    
    # Condition 2
    
    for ang in range(sg_ang_bin):
        ang_arr = rphi_meas[rphi_meas[:, 0] == ang, 1]
        
        if len(ang_arr) == 0:
            scan_grid[ang, :] = 0.3
           
            label_grid[ang, :] = 1
        else:
            min_r = np.min(ang_arr)
            scan_grid[ang, :min_r] = 0.3
            
            label_arr = rphi_meas[rphi_meas[:, 0] == ang, 2]
            label_grid[ang, :(min_r)] = 1

    ### Convert the spherical scan grid to the cartesian one
    ogm_sz = (ogm_shape[1], ogm_shape[0])
    ogm_cen = (int(ogm_shape[1] / 2), int(ogm_shape[0] / 2))
    radius = (MAX_RANGE / RESOLUTION) + SPHERICAL2CARTESIAN_BIAS
    ogm_step = cv2.warpPolar(scan_grid, ogm_sz, ogm_cen, radius, cv2.WARP_INVERSE_MAP)
    label_step = cv2.warpPolar(label_grid, ogm_sz, ogm_cen, radius, cv2.WARP_INVERSE_MAP)
    ogm_step[OOR_MASK] = 0.5
    label_step[OOR_MASK] = 0
    ogm_step = cv2.rotate(ogm_step, cv2.ROTATE_90_CLOCKWISE)
    label_step = cv2.rotate(label_step, cv2.ROTATE_90_CLOCKWISE)
    ogm_step = np.dstack([ogm_step, label_step])
    
    return ogm_step

In [ ]:
for item in vehicles:    
  print(item)    
  for  i in range(item[1]):
    lidar_path=item[0]
    save_path=item[2]
    if not os.path.exists(save_path):
      os.makedirs(save_path)

    if  os.path.exists(save_path+'/{}.npy'.format(i)):
            print("File {} is already created. Skipping item...".format(i))
            
            continue
    if not os.path.exists(lidar_path+'/{}.npy'.format(i)):
            print("File {} not found. Skipping item...".format(i))
            all_files_exist = False
            continue
    data=np.load(lidar_path+'/{}.npy'.format(i))
    lidar_raw=data.reshape(-1, 6)
      
      
    lidar_raw=lidar_raw[(lidar_raw[:, 2] <= -40) | (lidar_raw[:, 2] >= -37), :]


    # Define the radius (in meters)
    radius = 3

    # Calculate the Euclidean distance between each point in the point cloud and the lidar sensor position
    distances = np.sqrt((lidar_raw[:, 0])**2 + (lidar_raw[:, 1])**2)

    # Filter out all the points that are within the radius of the lidar sensor position
    lidar_raw = lidar_raw[distances > radius, :]



    intensity = np.array(lidar_raw[:, 3])
    labels=np.array(lidar_raw[:, 5]).astype(int)
    
    lidar_labeled=np.array(lidar_raw[..., [0, 1, 2, 5]])

    lidar_raw = lidar_raw[:, :3]

    road_filter=np.zeros_like(labels)
    road_filter[(labels == 0) | (labels == 7) | (labels == 6)] = 1
    
    reduced_labels=np.zeros_like(labels)
    reduced_labels[(labels == 7) | (labels == 6)] = 1
    reduced_labels[(labels == 1)| (labels == 2) | (labels == 11) | (labels == 19)] = 2
    reduced_labels[(labels == 3)| (labels == 15) | (labels == 17) | (labels == 18)] = 3
    reduced_labels[(labels == 5)| (labels == 18) | (labels == 19)] = 4
    reduced_labels[(labels == 9)] = 5
    reduced_labels[(labels == 4)| (labels == 10) | (labels == 20)] = 6
    reduced_labels[(labels == 8)| (labels == 14) | (labels == 22)] = 7
    reduced_labels[(labels == 21)] = 8  



    lidar_rlabeled = np.concatenate((lidar_raw, reduced_labels[..., np.newaxis]), axis=-1) 

    lidar_nonroad_labeled = filter_road_points(lidar_rlabeled,road_filter,0.1)  
    

    ### Initialize OGM
    ogm_time_0 = np.ones((MAP_SIZE_Y,MAP_SIZE_X,2)) 

    ### Only use the x-z axis of the point (ignore the height axis)
    lidar_ogm = lidar_nonroad_labeled[:,[0,1,3]]


    ogm_step = generate_semantic_ogm(lidar_ogm,ogm_time_0.shape)
    np.save(save_path+'/{}.npy'.format(i),ogm_step)
    print(save_path+'/{}.npy'.format(i))
    '''### Visualize
    fig,axs = plt.subplots(figsize=(12,6))
    plt.imshow(((1-ogm_step[:, :, 0])*255).astype(np.uint8),cmap='gray')
    plt.show()'''

('/content/drive/MyDrive/s_data/ground truth/scenario4_gt/Ego_1 vehicle.audi.a2/bird_lidar', 89, '/content/drive/MyDrive/s_data/ground truth/scenario4_gt/Ego_1 vehicle.audi.a2/bird_grid_gt')
/content/drive/MyDrive/s_data/ground truth/scenario4_gt/Ego_1 vehicle.audi.a2/bird_grid_gt/0.npy
/content/drive/MyDrive/s_data/ground truth/scenario4_gt/Ego_1 vehicle.audi.a2/bird_grid_gt/1.npy
/content/drive/MyDrive/s_data/ground truth/scenario4_gt/Ego_1 vehicle.audi.a2/bird_grid_gt/2.npy
/content/drive/MyDrive/s_data/ground truth/scenario4_gt/Ego_1 vehicle.audi.a2/bird_grid_gt/3.npy
/content/drive/MyDrive/s_data/ground truth/scenario4_gt/Ego_1 vehicle.audi.a2/bird_grid_gt/4.npy
/content/drive/MyDrive/s_data/ground truth/scenario4_gt/Ego_1 vehicle.audi.a2/bird_grid_gt/5.npy
/content/drive/MyDrive/s_data/ground truth/scenario4_gt/Ego_1 vehicle.audi.a2/bird_grid_gt/6.npy
/content/drive/MyDrive/s_data/ground truth/scenario4_gt/Ego_1 vehicle.audi.a2/bird_grid_gt/7.npy
/content/drive/MyDrive/s_data/gro